In [11]:
import pandas as pd

df = pd.read_excel(r"C:\Users\speec\OneDrive\Desktop\딥테크팁스\tips\output\병합.xlsx")
df.head()

,title,description,channel,published,videoId,viewCount,likeCount,commentCount,subscriberCount,text_raw,...,is_relevant,reach,R,engagement_rate,E,amplification_rate,A,S,L_influence,neg_ratio
0,skt 유심해킹이후 대처법 빨리하세요!,완전 난리난 skt 유심해킹이후 대처법 빨리하세요!,만렙백수,2025-04-26T12:11:40Z,rlaKrI4bjQs,5343089,91684,2319,87100,skt 유심해킹이후 대처법 빨리하세요! 완전 난리난 skt 유심해킹이후 대처법 빨리...,...,True,15.491315,1.000000,0.017593,0.089712,0.000434,0.036374,0.016750,0.435864,0.942857
1,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통),1분미만 #꿀팁 #유심 #유출 #설정 #정보.,1분미만,2025-04-26T08:19:53Z,3B6nv74tSkw,4727940,130773,5647,2570000,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통) 1분미만 ...,...,True,15.369000,0.992104,0.028854,0.147133,0.001194,0.100099,0.494231,0.510424,0.962963
2,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리,"SKT 해킹 사태가 터진 후 당일 소식과 초기 대응 방법을 커뮤니티에 공유했고, S...",ITSub잇섭,2025-04-28T10:30:42Z,yPFP6QcL_wI,3255259,80093,10251,2780000,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리 SKT 해킹 사...,...,True,14.995783,0.968012,0.027753,0.141520,0.003149,0.263916,0.534615,0.535906,0.965517
3,(중요)SKT안써도 당장 막으세요(해킹대처법),SKT #해킹 #유심 #대처방법 #인생2회차.,인생2회차,2025-04-29T08:00:02Z,1u4HSlfC4y0,2731172,35119,906,1050000,(중요)SKT안써도 당장 막으세요(해킹대처법) SKT #해킹 #유심 #대처방법 #인...,...,True,14.820242,0.956681,0.013190,0.067260,0.000332,0.027801,0.201923,0.428603,1.000000
4,SKT 유심 해킹이 엄청큰일인 이유,SKT 유심 해킹이 엄청큰일인 이유.,콩만이,2025-04-25T10:05:04Z,Po35L5hiAFw,2316993,20974,2171,36400,SKT 유심 해킹이 엄청큰일인 이유 SKT 유심 해킹이 엄청큰일인 이유.,...,True,14.655781,0.946064,0.009989,0.050937,0.000937,0.078527,0.007000,0.410112,0.964286


In [12]:
# Check for null values in the DataFrame
print("\nNull values in each column:")
print(df.isnull().sum())

# Get total number of null values
total_nulls = df.isnull().sum().sum()
print(f"\nTotal number of null values: {total_nulls}")
# neg_ratio 데이터가 널값이라는 것은 댓글이 모두 중립이라는 것 대부분분



Null values in each column:
title                  0
description           16
channel                0
published              0
videoId                0
viewCount              0
likeCount              0
commentCount           0
subscriberCount        0
text_raw               0
labels                 0
lexicon_hits           0
text_norm              0
is_relevant            0
reach                  0
R                      0
engagement_rate        0
E                      0
amplification_rate     0
A                      0
S                      0
L_influence            0
neg_ratio             23
dtype: int64

Total number of null values: 39


In [ ]:
# Drop rows where neg_ratio is null
df = df.dropna(subset=['neg_ratio'])

# Display DataFrame info
print("DataFrame info after dropping null values in neg_ratio:")
df.info()  
# 우리가 알고싶은 것은 부정적 댓글임으로 부정적 댓글만 남긴다. 



DataFrame info after dropping null values in neg_ratio:
<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 0 to 95
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               76 non-null     object 
 1   description         65 non-null     object 
 2   channel             76 non-null     object 
 3   published           76 non-null     object 
 4   videoId             76 non-null     object 
 5   viewCount           76 non-null     int64  
 6   likeCount           76 non-null     int64  
 7   commentCount        76 non-null     int64  
 8   subscriberCount     76 non-null     int64  
 9   text_raw            76 non-null     object 
 10  labels              76 non-null     object 
 11  lexicon_hits        76 non-null     object 
 12  text_norm           76 non-null     object 
 13  is_relevant         76 non-null     bool   
 14  reach               76 non-null     float64
 15  R       

In [5]:
from transformers import pipeline
classifier = pipeline("text-classification", model="snunlp/KR-FinBert-SC")

Device set to use cpu


In [6]:
# 게시글 감성 분석 (title + description 기준)
results = []
sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}

# 분석 대상: title + description
for idx, row in df[['title', 'description', 'videoId']].iterrows():
    try:
        # 텍스트 결합 및 길이 제한
        content = f"{row['title']} {row['description']}"
        if len(content) > 500:
            content = content[:500]
        
        result = classifier(content)
        results.append(result)
        label = result[0]['label']
        sentiment_counts[label] += 1
        
    except Exception as e:
        print(f"Error processing content {idx}: {str(e)}")
        # 에러 발생 시 중립 처리
        results.append([{'label': 'neutral', 'score': 0.0}])
        sentiment_counts['neutral'] += 1

# 결과 DataFrame 변환
content_results = pd.DataFrame({
    'videoId': df['videoId'],
    'title': df['title'],
    'description': df['description'],
    'content_sentiment': [r[0]['label'] for r in results],
    'confidence': [r[0]['score'] for r in results]
})

# 부정 콘텐츠 플래그 추가
content_results['is_negative'] = (content_results['content_sentiment'] == 'negative').astype(int)

# output 폴더 생성
import os
if not os.path.exists('output'):
    os.makedirs('output')

# 결과 저장
output_file = 'output/content_sentiment_results.xlsx'
content_results.to_excel(output_file, index=False)

# 결과 출력
print("\n📊 게시글 감성 분석 결과 요약 :")
print(f"👍 긍정: {sentiment_counts['positive']}개")
print(f"❗ 부정: {sentiment_counts['negative']}개")
print(f"😐 중립: {sentiment_counts['neutral']}개")
print(f"\n✅ 상세 결과 저장 완료: {output_file}")



📊 게시글 감성 분석 결과 요약 :
👍 긍정: 1개
❗ 부정: 44개
😐 중립: 31개

✅ 상세 결과 저장 완료: output/content_sentiment_results.xlsx


In [10]:
content_results

,videoId,title,description,content_sentiment,confidence,is_negative
0,rlaKrI4bjQs,skt 유심해킹이후 대처법 빨리하세요!,완전 난리난 skt 유심해킹이후 대처법 빨리하세요!,neutral,0.999339,0
1,3B6nv74tSkw,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통),1분미만 #꿀팁 #유심 #유출 #설정 #정보.,neutral,0.998684,0
2,yPFP6QcL_wI,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리,"SKT 해킹 사태가 터진 후 당일 소식과 초기 대응 방법을 커뮤니티에 공유했고, S...",negative,0.999730,1
3,1u4HSlfC4y0,(중요)SKT안써도 당장 막으세요(해킹대처법),SKT #해킹 #유심 #대처방법 #인생2회차.,neutral,0.623170,0
4,Po35L5hiAFw,SKT 유심 해킹이 엄청큰일인 이유,SKT 유심 해킹이 엄청큰일인 이유.,neutral,0.536251,0
...,...,...,...,...,...,...
94,olUXRGDDFzI,[자막뉴스] 유심 해킹은 &quot;중국 해커들 쓰는 수법&quot;…SKT가 내놓...,유심 해킹에 SKT는 45시간 만에 신고했습니다. 규정 위반입니다. [유영상/SKT...,negative,0.992742,1
95,K2EupFZTahI,SKT 위약금 면제 확정! 2달 만에 결정된 유심 해킹 피해 보상안! 추가 혜택 3...,아정당 대표번호 : 1833 - 3504],negative,0.837511,1
96,RR-UESo4AEQ,※긴급※ SKT 유심 해킹 대응! 내 명의 계좌 지금 확인하세요｜60대 이상 꼭 봐...,"최근 사회적으로 큰 이슈가 되고 있는 SK텔레콤 유심 정보 유출 사태에 대해, 60...",neutral,0.811169,0
97,L54T3gFfx0c,"🔴&#39;V3&#39; 안철수, SKT 유심 해킹에 &quot;무용지물 대책, 무...",SKT #유심 #대리점 #대란 #은전한닢 #유심한칩 #유심칩 #주세요 #구걸 #교체...,neutral,0.998462,0


In [ ]:
# 1. 콘텐츠 감정 점수 수치화 (-1 ~ 1)
def sentiment_to_score(row):
    if row['content_sentiment'] == 'positive':
        return row['confidence']
    elif row['content_sentiment'] == 'negative':
        return -row['confidence']
    else:
        return 0  # neutral

content_results['content_score'] = content_results.apply(sentiment_to_score, axis=1)

# 2. 콘텐츠 감정 점수 → 부정 비율 스케일로 변환 (0 ~ 1)
content_results['content_neg_ratio'] = (1 - content_results['content_score']) / 2

# 3. 댓글 부정 비율 데이터 (neg_stats)와 병합
# 중복 컬럼 처리를 위한 suffix 지정
content_results_filtered = content_results[['videoId', 'title', 'description', 'content_score', 'content_neg_ratio']]
merged = pd.merge(content_results_filtered, df, on='videoId', how='left', suffixes=('', '_y'))

# 중복된 컬럼 제거
columns_to_drop = [col for col in merged.columns if col.endswith('_y')]
merged = merged.drop(columns=columns_to_drop)

# 4. 감정 괴리도 계산
merged['polarity_gap'] = merged['content_neg_ratio'] - merged['neg_ratio']

# 5. 감정 충돌 여부 (중요 이슈 탐지)
merged['conflict_flag'] = (
    (merged['commentCount'] >= 30) &
    (merged['polarity_gap'].abs() >= 0.5)
)

# 6. 저장
from datetime import datetime
import os

if not os.path.exists('output'):
    os.makedirs('output')

final_file = f"output/final_merged_sentiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
merged.to_excel(final_file, index=False)

print("✅ 전체 통합 완료! 결과 파일 저장:", final_file)
# 최종 병합 파일 ( 댓글과 게시글의 감정분석이 들어간 데이터 필터링되어 총 76)


✅ 전체 통합 완료! 결과 파일 저장: output/final_merged_sentiment_20251020_135154.xlsx


In [2]:
import pandas as pd

# Read the sentiment analysis results file
sentiment_file = r"C:\Users\speec\OneDrive\Desktop\딥테크팁스\tips\output\final_merged_sentiment_20251020_135154.xlsx"
merged = pd.read_excel(sentiment_file)
print("✅ 감정분석 결과 파일을 불러왔습니다.")


✅ 감정분석 결과 파일을 불러왔습니다.


In [3]:
merged

,videoId,title,description,content_score,content_neg_ratio,channel,published,viewCount,likeCount,commentCount,...,R,engagement_rate,E,amplification_rate,A,S,L_influence,neg_ratio,polarity_gap,conflict_flag
0,rlaKrI4bjQs,skt 유심해킹이후 대처법 빨리하세요!,완전 난리난 skt 유심해킹이후 대처법 빨리하세요!,0.000000,0.500000,만렙백수,2025-04-26T12:11:40Z,5343089,91684,2319,...,1.000000,0.017593,0.089712,0.000434,0.036374,0.016750,0.435864,0.942857,-0.442857,False
1,3B6nv74tSkw,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통),1분미만 #꿀팁 #유심 #유출 #설정 #정보.,0.000000,0.500000,1분미만,2025-04-26T08:19:53Z,4727940,130773,5647,...,0.992104,0.028854,0.147133,0.001194,0.100099,0.494231,0.510424,0.962963,-0.462963,False
2,yPFP6QcL_wI,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리,"SKT 해킹 사태가 터진 후 당일 소식과 초기 대응 방법을 커뮤니티에 공유했고, S...",-0.999730,0.999865,ITSub잇섭,2025-04-28T10:30:42Z,3255259,80093,10251,...,0.968012,0.027753,0.141520,0.003149,0.263916,0.534615,0.535906,0.965517,0.034348,False
3,1u4HSlfC4y0,(중요)SKT안써도 당장 막으세요(해킹대처법),SKT #해킹 #유심 #대처방법 #인생2회차.,0.000000,0.500000,인생2회차,2025-04-29T08:00:02Z,2731172,35119,906,...,0.956681,0.013190,0.067260,0.000332,0.027801,0.201923,0.428603,1.000000,-0.500000,True
4,Po35L5hiAFw,SKT 유심 해킹이 엄청큰일인 이유,SKT 유심 해킹이 엄청큰일인 이유.,0.000000,0.500000,콩만이,2025-04-25T10:05:04Z,2316993,20974,2171,...,0.946064,0.009989,0.050937,0.000937,0.078527,0.007000,0.410112,0.964286,-0.464286,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,_YPs-K7WuDM,"[자막] SKT 유심 해킹, 위약금 면제 확답 못해 #유영상 #SKT",국회 #국회방송 #생중계 #과학기술정보방송통신위원회 #과방위 #YTN #방송 #통신...,-0.891282,0.945641,춘천MBC뉴스,2025-04-30T03:04:37Z,21286,146,129,...,0.643319,0.012919,0.065878,0.006060,0.507902,0.030385,0.381710,1.000000,-0.054359,False
72,u_kLir7D9dM,"SKT 유심 해킹, 결국 휴대폰 교체가 필요한 2가지 이유",안녕하세요 오늘도 SKT 유심 해킹 사태에 대한 영상을 긴급히 만들었습니다 구독자님...,-0.968017,0.984008,주터팬,2025-04-28T21:30:19Z,19618,479,124,...,0.638051,0.030737,0.156735,0.006321,0.529726,0.018423,0.410029,1.000000,-0.015992,False
73,kXYbk2vwQys,SKT 유심 해킹 생각보다 심각합니다! 유심교체나 타사로 옮기세요,skt해킹 #SK텔레콤해킹 #skt유심해킹 #심스와핑 #kt해킹 #lgu해킹 #유심...,0.000000,0.500000,삼척첫눈,2025-04-24T09:02:08Z,17813,241,106,...,0.631821,0.019480,0.099333,0.005951,0.498716,0.004058,0.382677,0.842105,-0.342105,False
74,K2EupFZTahI,SKT 위약금 면제 확정! 2달 만에 결정된 유심 해킹 피해 보상안! 추가 혜택 3...,아정당 대표번호 : 1833 - 3504],-0.837511,0.918755,아정당,2025-07-04T19:28:05Z,16301,138,36,...,0.626096,0.010674,0.054430,0.002208,0.185086,0.060962,0.309880,1.000000,-0.081245,False


merged.info()

In [4]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   videoId             76 non-null     object 
 1   title               76 non-null     object 
 2   description         65 non-null     object 
 3   content_score       76 non-null     float64
 4   content_neg_ratio   76 non-null     float64
 5   channel             76 non-null     object 
 6   published           76 non-null     object 
 7   viewCount           76 non-null     int64  
 8   likeCount           76 non-null     int64  
 9   commentCount        76 non-null     int64  
 10  subscriberCount     76 non-null     int64  
 11  text_raw            76 non-null     object 
 12  labels              76 non-null     object 
 13  lexicon_hits        76 non-null     object 
 14  text_norm           76 non-null     object 
 15  is_relevant         76 non-null     bool   
 16  reach     

In [5]:
def compute_final_issue_score(row):
    if row['commentCount'] >= 30:
        w_c, w_a = 0.3, 0.7
    else:
        w_c, w_a = 0.6, 0.4
    
    return w_c * row['content_neg_ratio'] + w_a * row['neg_ratio']
merged['final_negative_emotion_score'] = merged.apply(compute_final_issue_score, axis=1)
merged.head()

# 댓글이 30개 이상일 대 댓글 기반 부정비율을 높게 산정정

,videoId,title,description,content_score,content_neg_ratio,channel,published,viewCount,likeCount,commentCount,...,engagement_rate,E,amplification_rate,A,S,L_influence,neg_ratio,polarity_gap,conflict_flag,final_negative_emotion_score
0,rlaKrI4bjQs,skt 유심해킹이후 대처법 빨리하세요!,완전 난리난 skt 유심해킹이후 대처법 빨리하세요!,0.00000,0.500000,만렙백수,2025-04-26T12:11:40Z,5343089,91684,2319,...,0.017593,0.089712,0.000434,0.036374,0.016750,0.435864,0.942857,-0.442857,False,0.810000
1,3B6nv74tSkw,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통),1분미만 #꿀팁 #유심 #유출 #설정 #정보.,0.00000,0.500000,1분미만,2025-04-26T08:19:53Z,4727940,130773,5647,...,0.028854,0.147133,0.001194,0.100099,0.494231,0.510424,0.962963,-0.462963,False,0.824074
2,yPFP6QcL_wI,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리,"SKT 해킹 사태가 터진 후 당일 소식과 초기 대응 방법을 커뮤니티에 공유했고, S...",-0.99973,0.999865,ITSub잇섭,2025-04-28T10:30:42Z,3255259,80093,10251,...,0.027753,0.141520,0.003149,0.263916,0.534615,0.535906,0.965517,0.034348,False,0.975822
3,1u4HSlfC4y0,(중요)SKT안써도 당장 막으세요(해킹대처법),SKT #해킹 #유심 #대처방법 #인생2회차.,0.00000,0.500000,인생2회차,2025-04-29T08:00:02Z,2731172,35119,906,...,0.013190,0.067260,0.000332,0.027801,0.201923,0.428603,1.000000,-0.500000,True,0.850000
4,Po35L5hiAFw,SKT 유심 해킹이 엄청큰일인 이유,SKT 유심 해킹이 엄청큰일인 이유.,0.00000,0.500000,콩만이,2025-04-25T10:05:04Z,2316993,20974,2171,...,0.009989,0.050937,0.000937,0.078527,0.007000,0.410112,0.964286,-0.464286,False,0.825000


In [7]:
import os

if not os.path.exists('output'):
    os.makedirs('output')

merged.to_excel('output/Poc2_1.xlsx', index=False)
print("✅ 파일이 저장되었습니다: output/Poc2_1.xlsx")

✅ 파일이 저장되었습니다: output/Poc2_1.xlsx
